In [124]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [125]:
# 크롤링용
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib
import xmltodict
import json

In [126]:
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo'

# API Trial

In [127]:
# API 파싱 trial
queryParams = '?' + urlencode({ quote_plus('servicekey') : 'PpKgpE0UGRUPxpsO9SW9sYZAcg2pRiu%2B3Q1%2BS%2BDFGsTF4vwrQ64MSNJlkGPimqSvHbkqVINaX%2FWxHZTXq6j6cg%3D%3D',
    quote_plus('solYear') : '2015',
    quote_plus('solMonth') : '0' + str(5)})

request = urllib.request.Request(url+unquote(queryParams))
# print ('Your Request:\n'+url+queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read().decode('utf-8')
# print ("\nResult:")
# print (response_body)
# print ("\nDataType of Result Data:")
# print (type(response_body))

In [128]:
jsonString = xmltodict.parse(response_body)
jsonString

OrderedDict([('response',
              OrderedDict([('header',
                            OrderedDict([('resultCode', '00'),
                                         ('resultMsg', 'NORMAL SERVICE.')])),
                           ('body',
                            OrderedDict([('items',
                                          OrderedDict([('item',
                                                        [OrderedDict([('dateKind',
                                                                       '01'),
                                                                      ('dateName',
                                                                       '어린이날'),
                                                                      ('isHoliday',
                                                                       'Y'),
                                                                      ('locdate',
                                                                       '20150

In [129]:
int(jsonString['response']['body']['totalCount'])

2

In [130]:
lst = []

In [131]:
trial1501 = jsonString['response']['body']['items']['item']

In [132]:
trial1501

[OrderedDict([('dateKind', '01'),
              ('dateName', '어린이날'),
              ('isHoliday', 'Y'),
              ('locdate', '20150505'),
              ('seq', '1')]),
 OrderedDict([('dateKind', '01'),
              ('dateName', '석가탄신일'),
              ('isHoliday', 'Y'),
              ('locdate', '20150525'),
              ('seq', '1')])]

In [133]:
lst.append(trial1501)

In [134]:
lst

[[OrderedDict([('dateKind', '01'),
               ('dateName', '어린이날'),
               ('isHoliday', 'Y'),
               ('locdate', '20150505'),
               ('seq', '1')]),
  OrderedDict([('dateKind', '01'),
               ('dateName', '석가탄신일'),
               ('isHoliday', 'Y'),
               ('locdate', '20150525'),
               ('seq', '1')])]]

# 자동화

In [135]:
def api_parser(url, year, holi_list):
    for m in range(1,13):
        queryParams = '?' + urlencode({ quote_plus('servicekey') : 'PpKgpE0UGRUPxpsO9SW9sYZAcg2pRiu%2B3Q1%2BS%2BDFGsTF4vwrQ64MSNJlkGPimqSvHbkqVINaX%2FWxHZTXq6j6cg%3D%3D',
        quote_plus('solYear') : year,
        quote_plus('solMonth') : '0' + str(m) if m < 10 else str(m)})

        request = urllib.request.Request(url+unquote(queryParams))
        request.get_method = lambda: 'GET'
        response_body = urlopen(request).read().decode('utf-8')
        jsonString = xmltodict.parse(response_body)
        
        if int(jsonString['response']['body']['totalCount']) > 1 :
            for item in jsonString['response']['body']['items']['item'] :
                holi_list.append(item)
        elif int(jsonString['response']['body']['totalCount']) == 1:
            holi_list.append(jsonString['response']['body']['items']['item'])

In [136]:
holi17 = []
api_parser(url, '2017', holi17)
holi17_df = pd.DataFrame(holi17)

In [137]:
holi18 = []
api_parser(url, '2018', holi18)
holi18_df = pd.DataFrame(holi18)

In [138]:
holi19 = []
api_parser(url, '2019', holi19)
holi19_df = pd.DataFrame(holi19)

In [139]:
holi_df = pd.concat([holi17_df, holi18_df, holi19_df])
holi_df

/home/leebom/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,dateKind,dateName,isHoliday,locdate,remarks,seq
0,01,신정,Y,20170101,NaN,1
1,01,설날,Y,20170127,NaN,1
2,01,설날,Y,20170128,NaN,1
3,01,설날,Y,20170129,NaN,1
4,01,대체공휴일,Y,20170130,NaN,1
5,01,삼일절,Y,20170301,NaN,1
6,01,석가탄신일,Y,20170503,NaN,1
7,01,어린이날,Y,20170505,NaN,1
8,01,대통령선거일,Y,20170509,NaN,1
9,01,현충일,Y,20170606,NaN,1


In [140]:
holi_df = holi_df[holi_df['isHoliday'] == 'Y']
holi_df

,dateKind,dateName,isHoliday,locdate,remarks,seq
0,01,신정,Y,20170101,NaN,1
1,01,설날,Y,20170127,NaN,1
2,01,설날,Y,20170128,NaN,1
3,01,설날,Y,20170129,NaN,1
4,01,대체공휴일,Y,20170130,NaN,1
5,01,삼일절,Y,20170301,NaN,1
6,01,석가탄신일,Y,20170503,NaN,1
7,01,어린이날,Y,20170505,NaN,1
8,01,대통령선거일,Y,20170509,NaN,1
9,01,현충일,Y,20170606,NaN,1


In [141]:
holi_df.drop(['dateKind', 'remarks', 'seq', 'isHoliday'], axis=1, inplace=True)
holi_df

/home/leebom/.local/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,dateName,locdate
0,신정,20170101
1,설날,20170127
2,설날,20170128
3,설날,20170129
4,대체공휴일,20170130
5,삼일절,20170301
6,석가탄신일,20170503
7,어린이날,20170505
8,대통령선거일,20170509
9,현충일,20170606


In [142]:
holi_df.to_csv('./data/[회귀]all_공휴일.csv', index=False)